Emotion Classifier

# Setup

In [2]:
import pandas as pd

In [6]:
columns = ['text', 'emotion_labels', 'id']

go_emotions_train = pd.read_csv('../datasets/GoEmotions/train.tsv', sep='\t', header=None, names=columns)

go_emotions_val = pd.read_csv('../datasets/GoEmotions/dev.tsv', sep='\t', header=None, names=columns)

go_emotions_test= pd.read_csv('../datasets/GoEmotions/test.tsv', sep='\t', header=None, names=columns)


## Exploring Training Set

In [10]:
print("First 5 rows of the training set:\n")
display(go_emotions_train.head())

print("\n5 random rows from the training set:\n")
display(go_emotions_train.sample(5))

print("\nLast 5 rows from the training set:\n")
display(go_emotions_train.tail())



First 5 rows of the training set:



,text,emotion_labels,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj



5 random rows from the training set:



,text,emotion_labels,id
25191,I wish I could give it to you. I've always had...,8,eesk255
16014,Exactly! When the mom went to kekeing in [NAME...,"4,26",edd232r
13751,You must!!,27,ed9znbz
26112,yeh and the new ones just feel like reskin cod...,25,eeds40y
29034,Yeah ik that should be unacceptable too lmao,"1,4",efbot3b



Last 5 rows from the training set:



,text,emotion_labels,id
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8
43409,Enjoy the ride!,17,eecwmbq


In [11]:
go_emotions_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43410 entries, 0 to 43409
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            43410 non-null  object
 1   emotion_labels  43410 non-null  object
 2   id              43410 non-null  object
dtypes: object(3)
memory usage: 1017.5+ KB


In [17]:
# Split the 'emotion_labels' string by commas and count how many labels each row has
label_counts = go_emotions_train['emotion_labels'].str.split(',').apply(len)

# Get frequency of each count (1 label, 2 labels, etc.)
counts_distribution = label_counts.value_counts().sort_index()

print("Number of emotion labels per row:")
print(counts_distribution)


Number of emotion labels per row:
emotion_labels
1    36308
2     6541
3      532
4       28
5        1
Name: count, dtype: int64


In [18]:
from collections import Counter

# Flatten all label indices into a single list
all_labels = sum(go_emotions_train['emotion_labels'].str.split(',').tolist(), [])

# Count occurrences of each label index
label_freq = Counter(all_labels)

print("Counts for each emotion label index:")
for label, count in sorted(label_freq.items()):
    print(f"Label {label}: {count}")


Counts for each emotion label index:
Label 0: 4130
Label 1: 2328
Label 10: 2022
Label 11: 793
Label 12: 303
Label 13: 853
Label 14: 596
Label 15: 2662
Label 16: 77
Label 17: 1452
Label 18: 2086
Label 19: 164
Label 2: 1567
Label 20: 1581
Label 21: 111
Label 22: 1110
Label 23: 153
Label 24: 545
Label 25: 1326
Label 26: 1060
Label 27: 14219
Label 3: 2470
Label 4: 2939
Label 5: 1087
Label 6: 1368
Label 7: 2191
Label 8: 641
Label 9: 1269


In [19]:
# Normalize the label combinations by sorting them so '1,2' and '2,1' count as same combo
normalized_combos = go_emotions_train['emotion_labels'].apply(lambda x: ','.join(sorted(x.split(','))))

# Count occurrences of each unique combination
combo_freq = normalized_combos.value_counts()

print("Top 10 most common emotion label combinations:")
print(combo_freq.head(10))


Top 10 most common emotion label combinations:
emotion_labels
27    12823
0      2710
4      1873
15     1857
1      1652
3      1451
18     1427
10     1402
7      1389
2      1025
Name: count, dtype: int64


## Exploring Validation Set

## Exploring Test Set